<a href="https://colab.research.google.com/github/choons1k2/2022-Fall-Opensource-AI/blob/main/gender_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
from google.colab import drive
drive.mount('/content/drive')
f_path = '/content/heart.csv'

import pandas as pd
heart = pd.read_csv(f_path)


#OneHotEncoding
heart = pd.get_dummies(heart)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:


for i in range(20):

  Male = heart[ heart['Sex_M'] == 1 ]
  Female = heart[ heart['Sex_M'] == 0 ]
  Male = Male.sample(frac=0.266)
  


  heartTest = pd.concat([Male, Female])
  print(f"{i+1}. 남성 표본: {len(Male)}명, 여성 표본: {len(Female)}명")
  Positive = heartTest[ heartTest['HeartDisease'] == 1 ]
  Negative = heartTest[ heartTest['HeartDisease'] == 0 ]

  mPositive = len(Positive[Positive['Sex_M'] == 1])
  fPositive = len(Positive[Positive['Sex_M'] == 0])

  mNegative = len(Negative[Negative['Sex_M'] == 1])
  fNegative = len(Negative[Negative['Sex_M'] == 0])

  mRate = mPositive / (mPositive + mNegative)
  fRate = fPositive / (fPositive + fNegative)
  
  print(f"남성 양성률: {mRate}")
  print(f"여성 양성률: {fRate}\n")

1. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6010362694300518
여성 양성률: 0.25906735751295334

2. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6321243523316062
여성 양성률: 0.25906735751295334

3. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6683937823834197
여성 양성률: 0.25906735751295334

4. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6321243523316062
여성 양성률: 0.25906735751295334

5. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6113989637305699
여성 양성률: 0.25906735751295334

6. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6217616580310881
여성 양성률: 0.25906735751295334

7. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.5906735751295337
여성 양성률: 0.25906735751295334

8. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.616580310880829
여성 양성률: 0.25906735751295334

9. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6580310880829016
여성 양성률: 0.25906735751295334

10. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.5906735751295337
여성 양성률: 0.25906735751295334

11. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.5906735751295337
여성 양성률: 0.25906735751295334

12. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6113989637305699
여성 양성률: 0.259067357

In [84]:
Male = heart[ heart['Sex_M'] == 1 ]
Female = heart[ heart['Sex_M'] == 0 ]
Male = Male.sample(frac=0.266, random_state=123)
heartTest = pd.concat([Male, Female])


X = heartTest.drop('HeartDisease', axis=1)
y = (heartTest["HeartDisease"])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123)

X_train.shape


(308, 20)

In [85]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import BinaryAccuracy
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [86]:
model = Sequential()


model.add(Dense(units=20,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=60,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=30,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=1,activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(learning_rate=0.001),
              metrics=[BinaryAccuracy()])

In [88]:
model.fit(X_train, y_train, epochs = 500, 
          batch_size = 30, 
          validation_data = (X_test, y_test))

Epoch 1/500
11/11 [==============================] - 0s 12ms/step - loss: 0.3827 - binary_accuracy: 0.8442 - val_loss: 0.3585 - val_binary_accuracy: 0.8333
Epoch 2/500
11/11 [==============================] - 0s 6ms/step - loss: 0.3505 - binary_accuracy: 0.8636 - val_loss: 0.3810 - val_binary_accuracy: 0.7949
Epoch 3/500
11/11 [==============================] - 0s 8ms/step - loss: 0.3371 - binary_accuracy: 0.8766 - val_loss: 0.3875 - val_binary_accuracy: 0.7949
Epoch 4/500
11/11 [==============================] - 0s 7ms/step - loss: 0.3631 - binary_accuracy: 0.8539 - val_loss: 0.3810 - val_binary_accuracy: 0.7949
Epoch 5/500
11/11 [==============================] - 0s 8ms/step - loss: 0.4077 - binary_accuracy: 0.8117 - val_loss: 0.3762 - val_binary_accuracy: 0.8077
Epoch 6/500
11/11 [==============================] - 0s 5ms/step - loss: 0.3745 - binary_accuracy: 0.8409 - val_loss: 0.3601 - val_binary_accuracy: 0.8077
Epoch 7/500
11/11 [==============================] - 0s 5ms/step - lo

In [95]:

#Test with Male only
X2 = Male.drop('HeartDisease', axis=1)
y2 = (Male["HeartDisease"])

from sklearn.model_selection import train_test_split

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, 
                                                    test_size=0.2, 
                                                    random_state=123)
#test with only Male
model.fit(X_train, y_train, epochs = 500, 
          batch_size = 30, 
          validation_data = (X2_test, y2_test))

Epoch 1/500
6/6 [==============================] - 0s 24ms/step - loss: 0.4592 - binary_accuracy: 0.8571 - val_loss: 0.1926 - val_binary_accuracy: 0.9487
Epoch 2/500
6/6 [==============================] - 0s 10ms/step - loss: 0.3845 - binary_accuracy: 0.8636 - val_loss: 0.1758 - val_binary_accuracy: 0.9487
Epoch 3/500
6/6 [==============================] - 0s 13ms/step - loss: 0.3852 - binary_accuracy: 0.8571 - val_loss: 0.1775 - val_binary_accuracy: 0.9487
Epoch 4/500
6/6 [==============================] - 0s 12ms/step - loss: 0.3277 - binary_accuracy: 0.8766 - val_loss: 0.1689 - val_binary_accuracy: 0.9487
Epoch 5/500
6/6 [==============================] - 0s 17ms/step - loss: 0.4242 - binary_accuracy: 0.8442 - val_loss: 0.1692 - val_binary_accuracy: 0.9231
Epoch 6/500
6/6 [==============================] - 0s 14ms/step - loss: 0.3387 - binary_accuracy: 0.8831 - val_loss: 0.1860 - val_binary_accuracy: 0.9231
Epoch 7/500
6/6 [==============================] - 0s 15ms/step - loss: 0.36

In [98]:
#Test with Male only
X3 = Female.drop('HeartDisease', axis=1)
y3 = (Female["HeartDisease"])

from sklearn.model_selection import train_test_split

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, 
                                                    test_size=0.2, 
                                                    random_state=123)
#test with only Male
model.fit(X_train, y_train, epochs = 500, 
          batch_size = 30, 
          validation_data = (X3_test, y3_test))

Epoch 1/500
6/6 [==============================] - 0s 20ms/step - loss: 0.2555 - binary_accuracy: 0.9091 - val_loss: 0.2399 - val_binary_accuracy: 0.9231
Epoch 2/500
6/6 [==============================] - 0s 14ms/step - loss: 0.2711 - binary_accuracy: 0.9026 - val_loss: 0.2477 - val_binary_accuracy: 0.9231
Epoch 3/500
6/6 [==============================] - 0s 10ms/step - loss: 0.3122 - binary_accuracy: 0.9091 - val_loss: 0.2272 - val_binary_accuracy: 0.8974
Epoch 4/500
6/6 [==============================] - 0s 11ms/step - loss: 0.2944 - binary_accuracy: 0.8831 - val_loss: 0.2476 - val_binary_accuracy: 0.8718
Epoch 5/500
6/6 [==============================] - 0s 9ms/step - loss: 0.3006 - binary_accuracy: 0.9026 - val_loss: 0.2179 - val_binary_accuracy: 0.8718
Epoch 6/500
6/6 [==============================] - 0s 12ms/step - loss: 0.3273 - binary_accuracy: 0.8636 - val_loss: 0.2426 - val_binary_accuracy: 0.8974
Epoch 7/500
6/6 [==============================] - 0s 12ms/step - loss: 0.269